## Phase mappting: map Bloch sphere angles to notes

In [7]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, note, instrument

# Convert a qubit's reduced state to Bloch sphere angles
def state_to_bloch_angles(alpha, beta):
    theta = 2 * np.arccos(np.abs(alpha))
    phi = np.angle(beta) - np.angle(alpha)
    return theta, phi

# Convert Bloch angles to MIDI notes
def theta_to_note(theta):
    pitch = int((theta / np.pi) * 12)
    return pitch + 60  # Middle C (C4)

def phi_to_note(phi):
    pitch = int(((phi + np.pi) / (2 * np.pi)) * 12)
    return pitch + 72  # One octave above middle C

# Assign instruments to qubits
def qubit_to_instrument(index):
    if index == 0:
        return instrument.Flute()
    instruments = [instrument.Piano(), instrument.Violin(), instrument.Guitar(),
                   instrument.Clarinet(), instrument.Trumpet()]
    return instruments[index % len(instruments)]

# Apply gate to state using circuit reconstruction
def apply_gate(state, gate, qargs, num_qubits):
    temp_circuit = qiskit.QuantumCircuit(num_qubits)
    temp_circuit.append(gate, [temp_circuit.qubits.index(q) for q in qargs])
    return state.evolve(temp_circuit)

# Main function: quantum circuit to music
def circuit_to_music_explicit(circuit):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            # Record amplitudes at each barrier
            for i in range(num_qubits):
                # Use partial_trace to reduce the state for qubit i
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta))
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    # Convert each qubit's states into music
    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        for alpha, beta in qubit_states[i]:
            theta, phi = state_to_bloch_angles(alpha, beta)
            note_theta = note.Note(theta_to_note(theta), quarterLength=1)
            note_phi = note.Note(phi_to_note(phi), quarterLength=1)
            part.append([note_theta, note_phi])  # Play simultaneously
        music_score.append(part)

    return music_score, qubit_states

# Example: build a sample circuit
qc = qiskit.QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.barrier()
qc.rz(np.pi/4, 2)
qc.ry(np.pi/2, 1)
qc.rx(np.pi/3, 0)
qc.barrier()
qc.ccx(0, 1, 2)
qc.barrier()


# Generate the musical score
music_score, qubit_states = circuit_to_music_explicit(qc)

# Show the score or save it
music_score.show('text')  # Opens MIDI playback in music21
music_score.write('midi', fp='quantum_music_bloch.mid')  # Save to file if desired


{0.0} <music21.stream.Part 0x1157138c0>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.note.Note G#>
    {1.0} <music21.note.Note F#>
    {2.0} <music21.note.Note G#>
    {3.0} <music21.note.Note A>
    {4.0} <music21.note.Note G#>
    {5.0} <music21.note.Note C>
{6.0} <music21.stream.Part 0x115710890>
    {0.0} <music21.instrument.Violin 'Violin'>
    {0.0} <music21.note.Note G#>
    {1.0} <music21.note.Note F#>
    {2.0} <music21.note.Note G#>
    {3.0} <music21.note.Note D>
    {4.0} <music21.note.Note G#>
    {5.0} <music21.note.Note E>
{12.0} <music21.stream.Part 0x1152c17f0>
    {0.0} <music21.instrument.Guitar 'Guitar'>
    {0.0} <music21.note.Note C>
    {1.0} <music21.note.Note F#>
    {2.0} <music21.note.Note C>
    {3.0} <music21.note.Note F#>
    {4.0} <music21.note.Note F>
    {5.0} <music21.note.Note F#>


'quantum_music_bloch.mid'

## magnitude based mapping

In [8]:
def magnitude_to_note(magnitude, base_octave):
    # Map [0, 1] -> MIDI note within an octave (12 semitones)
    pitch = int(magnitude * 11)  # 0–11
    return 12 * base_octave + pitch

def circuit_to_music_magnitude(circuit):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    music_score = stream.Score()

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        music_score.append(part)

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]

                mag_alpha = np.abs(alpha)
                mag_beta = np.abs(beta)

                note_alpha = note.Note(magnitude_to_note(mag_alpha, base_octave=4), quarterLength=1)
                note_beta  = note.Note(magnitude_to_note(mag_beta, base_octave=5), quarterLength=1)

                music_score[i].append([note_alpha, note_beta])
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    return music_score


In [9]:
music_score = circuit_to_music_magnitude(qc)
music_score.show('text')  # Opens MIDI playback in music21
music_score.write('midi', fp='quantum_music_magnitude.mid')  # Save to file if desired

{0.0} <music21.stream.Part 0x1156d63f0>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.note.Note F>
    {1.0} <music21.note.Note C>
    {2.0} <music21.note.Note F>
    {3.0} <music21.note.Note C>
    {4.0} <music21.note.Note F>
    {5.0} <music21.note.Note D>
{0.0} <music21.stream.Part 0x1156d6120>
    {0.0} <music21.instrument.Violin 'Violin'>
    {0.0} <music21.note.Note F>
    {1.0} <music21.note.Note C>
    {2.0} <music21.note.Note F>
    {3.0} <music21.note.Note C>
    {4.0} <music21.note.Note F>
    {5.0} <music21.note.Note D>
{0.0} <music21.stream.Part 0x115713ce0>
    {0.0} <music21.instrument.Guitar 'Guitar'>
    {0.0} <music21.note.Note B->
    {1.0} <music21.note.Note C>
    {2.0} <music21.note.Note B->
    {3.0} <music21.note.Note C>
    {4.0} <music21.note.Note G#>
    {5.0} <music21.note.Note C>


'quantum_music_magnitude.mid'

## real/imaginary based mapping

In [10]:
def value_to_note(val, base_octave):
    # Map [-1, 1] to 0–11 then to note in an octave
    normalized = (val + 1) / 2  # now in [0,1]
    pitch = int(normalized * 11)
    return 12 * base_octave + pitch

def circuit_to_music_real_imag(circuit):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    music_score = stream.Score()

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        music_score.append(part)

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]

                notes = [
                    note.Note(value_to_note(np.real(alpha), base_octave=3), quarterLength=1),
                    note.Note(value_to_note(np.imag(alpha), base_octave=4), quarterLength=1),
                    note.Note(value_to_note(np.real(beta), base_octave=5), quarterLength=1),
                    note.Note(value_to_note(np.imag(beta), base_octave=6), quarterLength=1),
                ]

                music_score[i].append(notes)  # all 4 play together
        else:
            state = apply_gate(state, inst, qargs, num_qubits)

    return music_score

In [11]:
music_score = circuit_to_music_real_imag(qc)

music_score.write('midi', fp='quantum_music_re_imag.mid')  # Save to file if desired

'quantum_music_re_imag.mid'